<center>
<img src="../../img/ods_stickers.jpg">
## Открытый курс по машинному обучению
<center>
Автор материала: Юрий Кашницкий, программист-исследователь Mail.Ru Group <br> 

Материал распространяется на условиях лицензии [Creative Commons CC BY-NC-SA 4.0](https://creativecommons.org/licenses/by-nc-sa/4.0/). Можно использовать в любых целях (редактировать, поправлять и брать за основу), кроме коммерческих, но с обязательным упоминанием автора материала.

# <center>Домашнее задание № 10 (демо)
## <center> Прогнозирование задержек вылетов

Ваша задача – побить единственный бенчмарк в [соревновании](https://www.kaggle.com/c/flight-delays-2017) на Kaggle Inclass. Подробных инструкций не будет, будет только тезисно описано, как получен этот бенчмарк. Конечно, с помощью Xgboost. Надеюсь, на данном этапе курса вам достаточно бросить полтора взгляда на данные, чтоб понять, что это тот тип задачи, в которой затащит Xgboost. Но проверьте еще Catboost.

<img src='../../img/xgboost_meme.jpg' width=40% />

In [158]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.linear_model import LogisticRegression
from xgboost import XGBClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from sklearn.metrics import roc_auc_score
from sklearn.preprocessing import LabelEncoder

In [147]:
train = pd.read_csv('../data/flight_delays_train.csv')
test = pd.read_csv('../data/flight_delays_test.csv')

In [148]:
train.head()

,Month,DayofMonth,DayOfWeek,DepTime,UniqueCarrier,Origin,Dest,Distance,dep_delayed_15min
0,c-8,c-21,c-7,1934,AA,ATL,DFW,732,N
1,c-4,c-20,c-3,1548,US,PIT,MCO,834,N
2,c-9,c-2,c-5,1422,XE,RDU,CLE,416,N
3,c-11,c-25,c-6,1015,OO,DEN,MEM,872,N
4,c-10,c-7,c-6,1828,WN,MDW,OMA,423,Y


In [149]:
test.head()

,Month,DayofMonth,DayOfWeek,DepTime,UniqueCarrier,Origin,Dest,Distance
0,c-7,c-25,c-3,615,YV,MRY,PHX,598
1,c-4,c-17,c-2,739,WN,LAS,HOU,1235
2,c-12,c-2,c-7,651,MQ,GSP,ORD,577
3,c-3,c-25,c-7,1614,WN,BWI,MHT,377
4,c-6,c-6,c-3,1505,UA,ORD,STL,258


Итак, надо по времени вылета самолета, коду авиакомпании-перевозчика, месту вылета и прилета и расстоянию между аэропортами вылета и прилета предсказать задержку вылета более 15 минут. В качестве простейшего бенчмарка возьмем логистическую регрессию и два признака, которые проще всего взять: `DepTime` и `Distance`. У такой модели результат – 0.68202 на LB. 

In [150]:
X_train, y_train = train[['Distance', 'DepTime']].values, train['dep_delayed_15min'].map({'Y': 1, 'N': 0}).values
X_test = test[['Distance', 'DepTime']].values

X_train_part, X_valid, y_train_part, y_valid = train_test_split(X_train, 
                                                                y_train, 
                                                                test_size=0.3, 
                                                                random_state=17)

In [151]:
logit = LogisticRegression(random_state=17)

logit.fit(X_train_part, y_train_part)
logit_valid_pred = logit.predict_proba(X_valid)[:, 1]
roc_auc_score(y_valid, logit_valid_pred)

/Users/ilchenkoslava/anaconda3/lib/python3.5/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


0.67897337310137207

In [152]:
logit.fit(X_train, y_train)
logit_test_pred = logit.predict_proba(X_test)[:, 1]

pd.Series(logit_test_pred, 
          name='dep_delayed_15min').to_csv('logit_2feat.csv', 
                                           index_label='id', 
                                           header=True)

/Users/ilchenkoslava/anaconda3/lib/python3.5/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


Как был получен бенчмарк в соревновании:
- Признаки `Distance` и  `DepTime` брались без изменений
- Создан признак "маршрут" из исходных `Origin` и `Dest`
- К признакам `Month`, `DayofMonth`, `DayOfWeek`, `UniqueCarrier` и "маршрут" применено OHE-преобразование (`LabelBinarizer`)
- Выделена отложенная выборка
- Обучалась логистическая регрессия и градиентный бустинг (xgboost), гиперпараметры бустинга настраивались на кросс-валидации, сначала те, что отвечают за сложность модели, затем число деревьев фиксировалось равным 500 и настраивался шаг градиентного спуска
- С помощью `cross_val_predict` делались прогнозы обеих моделей на кросс-валидации (именно предсказанные вероятности), настраивалась линейная смесь ответов логистической регрессии и градиентного бустинга вида $w_1 * p_{logit} + (1 - w_1) * p_{xgb}$, где $p_{logit}$ – предсказанные логистической регрессией вероятности класса 1, $p_{xgb}$ – аналогично. Вес $w_1$ подбирался вручную. 
- В качестве ответа для тестовой выборки бралась аналогичная комбинация ответов двух моделей, но уже обученных на всей обучающей выборке.

Описанный план ни к чему не обязывает – это просто то, как решение получил автор задания. Возможно, мы не захотите следовать намеченному плану, а добавите, скажем, пару хороших признаков и обучите лес из тысячи деревьев.

Удачи!

## DecisionTreeClassifier 

In [159]:
train.head()

,Month,DayofMonth,DayOfWeek,DepTime,UniqueCarrier,Origin,Dest,Distance,dep_delayed_15min
0,c-8,c-21,c-7,1934,AA,ATL,DFW,732,N
1,c-4,c-20,c-3,1548,US,PIT,MCO,834,N
2,c-9,c-2,c-5,1422,XE,RDU,CLE,416,N
3,c-11,c-25,c-6,1015,OO,DEN,MEM,872,N
4,c-10,c-7,c-6,1828,WN,MDW,OMA,423,Y


In [178]:
data_train = train
data_test = test

labels = [label for label in data.columns if label not in ['DepTime', 'Distance']]
label_encoder = LabelEncoder()

for label in labels:
    data_train[label] = label_encoder.fit_transform(data_train[label])
    if label != 'dep_delayed_15min':
        data_test[label] = label_encoder.fit_transform(data_test[label])

data_train.head()

,Month,DayofMonth,DayOfWeek,DepTime,UniqueCarrier,Origin,Dest,Distance,dep_delayed_15min
0,10,13,6,1934,0,18,78,732,0
1,6,12,2,1548,18,217,171,834,0
2,11,11,4,1422,20,228,59,416,0
3,2,17,5,1015,15,78,175,872,0
4,1,28,5,1828,19,174,199,423,1


In [180]:
X_train, y_train = data.drop(['dep_delayed_15min'], axis=1).values, data['dep_delayed_15min'].values
X_test = data_test.values

X_train_part, X_valid, y_train_part, y_valid = train_test_split(X_train, 
                                                                y_train, 
                                                                test_size=0.3, 
                                                                random_state=17)

In [181]:
%%time 
train_tree = DecisionTreeClassifier(random_state=17)
tree_params = {'max_depth': range(2,11)}

best_tree = GridSearchCV(train_tree, tree_params, cv=5, n_jobs=-1)                   

best_tree.fit(X_train_part, y_train_part)

CPU times: user 360 ms, sys: 306 ms, total: 667 ms
Wall time: 4.15 s


In [183]:
print("Best params:", best_tree.best_params_)

Best params: {'max_depth': 6}


In [184]:
tuned_tree = DecisionTreeClassifier(max_depth=locally_best_tree.best_params_['max_depth'], random_state=17)
tuned_tree.fit(X_train_part, y_train_part)

DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=6,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=17,
            splitter='best')

In [185]:
print('Результат -', roc_auc_score(y_valid, tuned_tree.predict_proba(X_valid)[:, 1]))

Результат - 0.694564430624


## RandomForestClassifier

In [208]:
from sklearn.model_selection import StratifiedKFold

rf1 = RandomForestClassifier(n_estimators=100, n_jobs=-1, 
                            random_state=17, oob_score=True, 
                            class_weight='balanced')

skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=5)

In [209]:
%%time

rf = RandomForestClassifier(n_estimators=100, random_state=17)
forest_params = {'max_depth': range(10, 21),
                'max_features': range(1, 9)}

best_forest_grid = GridSearchCV(rf1, forest_params, cv=skf, n_jobs=-1, verbose=True)
best_forest_grid.fit(X_train_part, y_train_part)

Fitting 5 folds for each of 88 candidates, totalling 440 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed:  1.2min
[Parallel(n_jobs=-1)]: Done 184 tasks      | elapsed:  6.2min
[Parallel(n_jobs=-1)]: Done 440 out of 440 | elapsed: 17.6min finished


CPU times: user 32.6 s, sys: 1.45 s, total: 34 s
Wall time: 17min 43s


In [210]:
print("Best params:", best_forest_grid.best_params_)

Best params: {'max_depth': 20, 'max_features': 6}


In [213]:
%%time
best_rf = RandomForestClassifier(n_estimators=500, max_depth=best_forest_grid.best_params_['max_depth'],
                                 max_features=best_forest_grid.best_params_['max_features'],
                                 random_state=17, n_jobs=-1)
best_rf.fit(X_train_part, y_train_part)

CPU times: user 2min 35s, sys: 1.08 s, total: 2min 36s
Wall time: 20.4 s


In [214]:
print('Результат -', roc_auc_score(y_valid, best_rf.predict_proba(X_valid)[:, 1]))

Результат - 0.730963636255


In [215]:
best_rf.fit(X_train, y_train)
best_rf_pred = best_rf.predict_proba(X_test)[:, 1]

pd.Series(best_rf_pred, 
          name='dep_delayed_15min').to_csv('rf_6feat.csv', 
                                           index_label='id', 
                                           header=True)